<a href="https://colab.research.google.com/github/JacobyoonDH/GAMA-Floodsim-BagongSilangan/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> ### EEE6503-01: Computer Vision

# Assignment \# 2: Mask R-CNN

**<div style="text-align: right"> Due date: Nov. 1st, 2024. </div>**
**<div style="text-align: right"> Please upload your file @ learnus by 9:00 PM. </div>**
<div style="text-align: right"> Lee Geon(이건): <a href="mailto:geon.lee@yonsei.ac.kr">geon.lee@yonsei.ac.kr</a>
<div style="text-align: right"> Sanghoon Lee(이상훈): <a href="mailto:shoon.lee@yonsei.ac.kr">shoon.lee@yonsei.ac.kr</a> </div>

### *Assignment Instructions:*
1. Write a program implementing a particular algorithm to solve a given problem.
2. You can use both Korean and English for your report.
3. **Analyze the algorithm, theoretically and empirically.**
4. **Report your results.**

<h2><span style="color:blue">[2024314217] [윤두휘]</span> </h2>

In [1]:
import datetime
print("This code is written at " + str(datetime.datetime.now()))

This code is written at 2024-10-25 04:20:53.918390


The objective of this assignment is to fine-tune a [Mask R-CNN](https://arxiv.org/abs/1703.06870) model using a [PennFudanPed](https://www.cis.upenn.edu/~jshi/ped_html/) dataset.

Do not import other functions using "wget", unless otherwise specified.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import os

torch.manual_seed(100)
np.random.seed(100)

In [3]:
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")

0

In [ ]:
#!wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip -P data
!cd ./data && unzip PennFudanPed.zip


Archive:  PennFudanPed.zip
replace PennFudanPed/added-object-list.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Step 1: Load a dataset

You can adjust the batch size if your GPU memory is not enough.

In [5]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = read_image(img_path)
        mask = read_image(mask_path)
        # instances are encoded as different colors
        obj_ids = torch.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        num_objs = len(obj_ids)

        # split the color-encoded mask into a set
        # of binary masks
        masks = (mask == obj_ids[:, None, None]).to(dtype=torch.uint8)

        # get bounding box coordinates for each mask
        boxes = masks_to_boxes(masks)

        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = idx
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Wrap sample and targets into torchvision tv_tensors:
        img = tv_tensors.Image(img)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=F.get_size(img))
        target["masks"] = tv_tensors.Mask(masks)
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [6]:
from torchvision.transforms import v2 as T

def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

In [7]:
import utils

dataset = PennFudanDataset('data/PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=utils.collate_fn
)

FileNotFoundError: [Errno 2] No such file or directory: 'data/PennFudanPed/PNGImages'

In [ ]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2
# use our dataset and defined transformations
dataset = PennFudanDataset('data/PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('data/PennFudanPed', get_transform(train=False))

# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=utils.collate_fn
)


# use our dataset and defined transformations
dataset = PennFudanDataset('data/PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('data/PennFudanPed', get_transform(train=False))

# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=utils.collate_fn
)

Step 2: Load a model

Load a Mask R-CNN model with a backbone of FPN pretrained on a COCO dataset.

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )

    return model

# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background

model = get_model_instance_segmentation(num_classes)

Write your own MultiScaleRoIAlign code in "model.roi_heads.box_roi_pool", instead of using **torchvision.ops.MultiScaleRoIAlign , torchvision.ops.RoIAlign, and torchvision.ops.roi_align**.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomMultiScaleRoIAlign(nn.Module):
    def __init__(self, output_size, sampling_ratio=2):
        super(CustomMultiScaleRoIAlign, self).__init__()
        self.output_size = output_size
        self.sampling_ratio = sampling_ratio

    def forward(self, features, proposals, image_shapes):
        # This implementation assumes features from multiple levels and combines them
        # features: list of tensors with different spatial sizes
        pooled_features = []
        for proposal in proposals:
            # Determine appropriate feature map based on the scale of the proposal
            # Here we use a simple heuristic: select the feature map based on proposal size
            best_level = self.select_feature_map_level(proposal, features)
            feature_map = features[best_level]

            # Extract features for the current proposal
            roi_feature = self.roi_pool(feature_map, proposal)
            pooled_features.append(roi_feature)

        # Stack all pooled features into a single tensor
        pooled_features = torch.stack(pooled_features, dim=0)
        return pooled_features

    def select_feature_map_level(self, proposal, features):
        # Heuristic to select feature level based on proposal size
        # Adjust this heuristic to suit your requirement
        area = (proposal[2] - proposal[0]) * (proposal[3] - proposal[1])
        if area < 32 * 32:
            return 0
        elif area < 64 * 64:
            return 1
        elif area < 128 * 128:
            return 2
        else:
            return 3

    def roi_pool(self, feature_map, proposal):
        # Extract a region of interest from the feature map and perform adaptive max pooling
        x1, y1, x2, y2 = proposal.int()
        roi = feature_map[:, :, y1:y2, x1:x2]
        pooled_roi = F.adaptive_max_pool2d(roi, self.output_size)
        return pooled_roi


In [ ]:
print(model.roi_heads.box_roi_pool)

In [ ]:
model.roi_heads.box_roi_pool = CustomMultiScaleRoIAlign(output_size=(7, 7))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

Step 3: Train the model

Configurations for training the model is shown below. Please refer to [paper](https://arxiv.org/abs/1703.06870) for loss functions.

* Optimization: Stochastic gradient descent (SGD)
* Learning rate: 0.005 (You can change the learning rate)
* The number of epochs: 5 (You can change the number of epochs)

In [ ]:
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

In [ ]:
# Train the model
import time

# Define the number of epochs and device
epochs = 5
model.train()

for epoch in range(epochs):
    start_time = time.time()
    epoch_loss = 0.0

    for images, targets in data_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Zero out the gradient
        optimizer.zero_grad()

        # Forward pass
        loss_dict = model(images, targets)

        # Total loss calculation
        losses = sum(loss for loss in loss_dict.values())

        # Backward pass and optimizer step
        losses.backward()
        optimizer.step()

        # Accumulate epoch loss
        epoch_loss += losses.item()

    # Update learning rate
    lr_scheduler.step()

    end_time = time.time()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Time: {end_time - start_time:.2f} sec")

Step 4: Test the model

Evaluate the detection performance using the test_loader without using **torchvision.ops.box_iou**. Write your own evalutaion code.
* Box-level mAP: a mean value of Average Precision (AP) @ [box IoU=0.50:0.05:0.95].
* Mask-level mAP: a mean value of Average Precision (AP) @ [mask IoU=0.50:0.05:0.95].

In [ ]:
# write your own code for computing IoU(Intersection over union)
# given ground-truth bounding box and predicted box.
def compute_bbox_IoU(gt_bbox, pred_bbox):
    #############
    # Code here #
    #############

# write your own code for computing IoU(Intersection over union)
# given ground-truth mask and predicted mask.
def compute_mask_IoU(gt_mask, pred_mask):
    #############
    # Code here #
    #############

# write your own code for computing AP(Average Precision)
# given ground-truth bounding boxes, predicted boxes, and scores of predicted boxes.
def compute_bbox_AP(gt_bboxes, pred_bboxes, scores, iou_threshold)
    #############
    # Code here #
    #############

# write your own code for computing AP(Average Precision)
# given ground-truth mask, predicted mask, and scores of maskes.
def compute_mask_AP(gt_maskes, pred_maskes, scores, iou_threshold)
    #############
    # Code here #
    #############

In [ ]:
## Test the model

#############
# Code here #
#############

Step 5: Visualization

Visualize an example image along with corresponding box and mask predictions.

In [ ]:
## Visualize an example of predictions

#############
# Code here #
#############

Step 6: Discussion

Train and evaluate the model by replacing the ROIAlign operator with the ROIPool operator. Compare the mAP results without using **torchvision.ops.RoIPool, and torchvision.ops.roi_pool**.

<!-- Step 6: Discussion

Train and evaluate the detection performance of the model by replacing the ROIAlign operator with the ROIPool operator. Compare the results without using **torchvision.ops.RoIPool, and torchvision.ops.roi_pool**. -->